In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nlp-getting-started/train.csv
/kaggle/input/nlp-getting-started/test.csv
/kaggle/input/nlp-getting-started/sample_submission.csv


In [2]:
df_train = pd.read_csv("/kaggle/input/nlp-getting-started/train.csv")
df_test = pd.read_csv("/kaggle/input/nlp-getting-started/test.csv")

In [3]:
df_train

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


# Cleaning

In [4]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
cleaned_tweet=[]
for i in range(0,len(df_train)):
    tweet=re.sub('[^a-zA-Z]',' ',df_train['text'][i])
    tweet=tweet.lower()
    tweet=tweet.split()
    ps=PorterStemmer()
    tweet=[ps.stem(word) for word in tweet if not word in set(stopwords.words('english'))]
    tweet=' '.join(tweet)
    cleaned_tweet.append(tweet)

In [6]:
cleaned_tweet_test=[]
for i in range(0,len(df_test)):
    tweet=re.sub('[^a-zA-Z]',' ',df_test['text'][i])
    tweet=tweet.lower()
    tweet=tweet.split()
    ps=PorterStemmer()
    tweet=[ps.stem(word) for word in tweet if not word in set(stopwords.words('english'))]
    tweet=' '.join(tweet)
    cleaned_tweet_test.append(tweet)

In [7]:
print(cleaned_tweet_test)

['happen terribl car crash', 'heard earthquak differ citi stay safe everyon', 'forest fire spot pond gees flee across street cannot save', 'apocalyps light spokan wildfir', 'typhoon soudelor kill china taiwan', 'shake earthquak', 'probabl still show life arsen yesterday eh eh', 'hey', 'nice hat', 'fuck', 'like cold', 'nooooooooo', 'tell', '', 'awesom', 'birmingham wholesal market ablaz bbc news fire break birmingham wholesal market http co irwqcezweu', 'sunkxssedharri wear short race ablaz', 'previouslyondoyintv toke makinwa marriag crisi set nigerian twitter ablaz http co cmghxba xi', 'check http co roi nsmejj http co tj zjin http co yduixefip http co lxtjc kl nsfw', 'psa split person techi follow ablaz co burner follow ablaz', 'bewar world ablaz sierra leon amp guap', 'burn man ablaz turban diva http co hodwosamw via etsi', 'diss song peopl take thing run smh eye open though set game ablaz cyhitheprync', 'rape victim die set ablaz year old girl die burn injuri set ablaz http co uk hn

# BagOfWords

In [8]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer()
X = cv.fit_transform(cleaned_tweet)
test_vectors = cv.transform(cleaned_tweet_test)
y=df_train['target']

# Cross Validation

In [9]:
from sklearn import feature_extraction, linear_model, model_selection, preprocessing
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

# Model

In [10]:
classifier = linear_model.RidgeClassifier()
classifier.fit(X_train, y_train)

RidgeClassifier()

In [11]:
y_pred = classifier.predict(X_test)
y_pred

array([0, 0, 0, ..., 1, 0, 0])

In [12]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[758 128]
 [223 414]]


0.7695338148391333

# Working on Test data

In [13]:
y_pred_test = classifier.predict(test_vectors)
y_pred_test

array([0, 1, 1, ..., 1, 1, 0])

In [14]:
testpred=pd.DataFrame(y_pred_test)

In [15]:
sub_df=pd.read_csv("/kaggle/input/nlp-getting-started/sample_submission.csv")
datasets=pd.concat([sub_df['id'],testpred],axis=1)
datasets.columns=['id','target']
datasets.to_csv('NewSubmission.csv',index=False)